### H-Pile Design 
#### Per NHI Courses No. 132021 and 132022 "Design and Construction of Driven Pile Foundations - Volume 1 & 2"
####     "LRFD Bridge Design Specifications" 9th Edition 2020 

In [3]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Button
from IPython.display import display
from ipyfilechooser import FileChooser
import io
import json

c:\Users\millern8989\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Pile Structural Information

In [8]:
#load csv with all the H-Pile info
H_Pile = pd.read_csv("https://raw.githubusercontent.com/geotechnick/Geotechnical_Design/main/Pile_Design/H_Piles/HP_Spec_Table.csv")
Fy = 50 # Pile yield stress in ksi
pile_names = H_Pile['Section_E'].unique().tolist()

#filter out the metric values
H_Pile = H_Pile.loc[:, ~H_Pile.columns.str.endswith('_M')]

#create drop down with the pile names 
dropdown = widgets.Dropdown(options=pile_names, description='Select Value:')

# Function to display the selected row 
def display_row(selected_value):
    global Pile_Info
    Pile_Info = H_Pile[H_Pile['Section_E'] == selected_value] #creates dataframe that only has the selected value in it
    pile_units = H_Pile[H_Pile['Section_E'] == 'Units_English'] #create separate dataframe that has units
    pile_display = pd.concat([Pile_Info, pile_units], ignore_index=True) #combine pile properties and pile units into single dataframe
    
    ## adds case_1 to the data frame
    # Get the current index label of the first row
    old_index_label = Pile_Info.index[0]

    # Define the new label for the first row
    new_index_label = 'case_1'

    # Rename the first row
    Pile_Info = Pile_Info.rename(index={old_index_label: new_index_label})

    # Now the first row is renamed and shifted
    Pile_Info.reset_index(inplace=True)

    display(pile_display.T)
# Connect the dropdown widget to the display function using interact
widgets.interact(display_row, selected_value=dropdown) 

interactive(children=(Dropdown(description='Select Value:', options=('Units_English', 'HP 8x36', 'HP 10x42', '…

<function __main__.display_row(selected_value)>

#### Upload Soil Profile

In [9]:
# Function to handle file upload
def handle_upload(change):
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    content_bytes = io.BytesIO(content)
    df = pd.read_csv(content_bytes)

    # For now, let's just print the first few rows
    print(df.head())
    # Save the dataframe as a global variable so it can be used outside of the function 
    global Layer_Properties
    Layer_Properties = df

# Creating file upload widget
file_upload = widgets.FileUpload(accept='.csv')

# Attaching the event handler
file_upload.observe(handle_upload, names='value')

# Displaying the widget
display(file_upload)

FileUpload(value=(), accept='.csv', description='Upload')

                layer non_soil  lv_top_elev  mlv_top_elev  hv_top_elev  \
0  Design_Water_Level        y          8.5           9.5         10.5   
1             Layer I      NaN         10.5          10.5         10.5   
2       Layer II Clay      NaN         -1.0           0.0          1.0   
3           Layer III      NaN         -6.0          -5.0         -4.0   
4       Layer IV Clay      NaN        -32.0         -31.0        -30.0   

   lv_bottom_elev  mlv_bottom_elev  hv_bottom_elev  lv_top_depth  \
0             NaN              NaN             NaN           NaN   
1            -1.0              0.0             1.0           NaN   
2            -6.0             -5.0            -4.0           NaN   
3           -32.0            -31.0           -30.0           NaN   
4           -47.0            -46.0           -45.0           NaN   

   mlv_top_depth  ...  lv_residual_coh  mlv_residual_coh  hv_residual_coh  \
0            NaN  ...              NaN               NaN             

In [10]:
#Should add Pile_Axial_Info with ground_surface, top_pile, pile_length, predrill_depth, steel_strength
# Define the widgets for user input
ground_surface_text = widgets.Text(description="Ground Surface:")
top_pile_text = widgets.Text(description="Top Pile:")
pile_length_float = widgets.FloatText(description="Pile Length:")
predrill_depth_float = widgets.FloatText(description="Predrill Depth:")
steel_strength_float = widgets.FloatText(description="Steel Strength:")

# Function to handle user input and create DataFrame
def create_dataframe(ground_surface, top_pile, pile_length, predrill_depth, steel_strength):
    global Pile_Axial_Info
    data = {
        "Ground Surface": [ground_surface],
        "Top Pile": [top_pile],
        "Pile Length": [pile_length],
        "Predrill Depth": [predrill_depth],
        "Steel Strength": [steel_strength]
    }
    Pile_Axial_Info = pd.DataFrame(data)
    
    ## adds case_1 to the data frame
    # Get the current index label of the first row
    old_index_label = Pile_Axial_Info.index[0]

    # Define the new label for the first row
    new_index_label = 'case_1'

    # Rename the first row
    Pile_Axial_Info = Pile_Axial_Info.rename(index={old_index_label: new_index_label})

    # Now the first row is renamed and shifted
    Pile_Axial_Info.reset_index(inplace=True)

    return Pile_Axial_Info

# Button to trigger dataframe creation
submit_button = widgets.Button(description="Submit")

def on_submit_button_clicked(b):
    display(create_dataframe(ground_surface_text.value, top_pile_text.value, pile_length_float.value, predrill_depth_float.value, steel_strength_float.value))

submit_button.on_click(on_submit_button_clicked)

# Display the widgets and button
input_widgets = widgets.VBox([ground_surface_text, top_pile_text, pile_length_float, predrill_depth_float, steel_strength_float, submit_button])
display(input_widgets)

,index,Ground Surface,Top Pile,Pile Length,Predrill Depth,Steel Strength
0,case_1,0,1,100.0,5.0,50.0


### Save properties to JSON

In [11]:
def save_json_to_file(json_data, file_path):
    if not file_path.endswith('.json'):
        file_path += '.json'
    with open(file_path, 'w') as json_file:
        json_file.write(json_data)
    print(f"JSON data saved to {file_path}")

def save_json_widget(json_data):
    file_chooser = FileChooser()
    file_chooser.use_dir_icons = True
    file_chooser.title = "Select a location to save the JSON file"
    save_button = widgets.Button(description="Save JSON")

    def save_json(button):
        if file_chooser.selected:
            save_json_to_file(json_data, file_chooser.selected)
        else:
            print("Please select a valid file path.")

    save_button.on_click(save_json)
    return widgets.VBox([file_chooser, save_button])

# Create JSON File ###############################
# Convert DataFrames to dictionaries
dict1 = Pile_Info.to_dict(orient='records')
dict2 = Layer_Properties.to_dict(orient='records')
dict3 = Pile_Axial_Info.to_dict(orient='records')

# Create a dictionary with dictionaries
pile_dict = {'Pile_Info': dict1, 'Layer_Properties': dict2, 'Pile_Axial_Info': dict3}

# Convert the dictionary to JSON format
json_data = json.dumps(pile_dict, indent=4)
###################################################

# Display the widget
save_json_widget(json_data)

JSON data saved to C:\Users\millern8989\OneDrive - ARCADIS\Documents\Python Calculations\Pile_Save.json


### Upload from save file


In [12]:
def handle_upload(change):
    global Pile_Info
    global Layer_Properties
    global Pile_Axial_Info
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    content_bytes = content.tobytes()
    content_str = content_bytes.decode('utf-8')
    data = json.loads(content_str)

    # Check if 'Pile_Info' key exists and create a DataFrame
    if 'Pile_Info' in data:
        Pile_Info = pd.DataFrame(data['Pile_Info'])
        print("DataFrame for 'Pile_Info':")
        print(Pile_Info)
        # Optionally, you can save the DataFrame to a file
        # pile_info_df.to_csv("pile_info_data.csv", index=False)
    else:
        print("The key 'Pile_Info' does not exist in the JSON data.")

    # Check if 'Soil_Properties' key exists and create a DataFrame
    if 'Layer_Properties' in data:
        Layer_Properties = pd.DataFrame(data['Layer_Properties'])
        print("\nDataFrame for 'Layer_Properties':")
        print(Layer_Properties)
        # Optionally, you can save the DataFrame to a file
        # soil_properties_df.to_csv("soil_properties_data.csv", index=False)
    else:
        print("The key 'Layer_Properties' does not exist in the JSON data.")

        # Check if 'Soil_Properties' key exists and create a DataFrame
    if 'Pile_Axial_Info' in data:
        Pile_Axial_Info = pd.DataFrame(data['Pile_Axial_Info'])
        print("\nDataFrame for 'Pile_Axial_Info':")
        print(Pile_Axial_Info)
        # Optionally, you can save the DataFrame to a file
        # soil_properties_df.to_csv("soil_properties_data.csv", index=False)
    else:
        print("The key 'Pile_Axial_Info' does not exist in the JSON data.")

# Creating file upload widget
file_upload = widgets.FileUpload(accept='.json')

# Attaching the event handler
file_upload.observe(handle_upload, names='value')

# Displaying the widget
display(file_upload)

FileUpload(value=(), accept='.json', description='Upload')

In [5]:
class pile_design:
    
    def __init__(self):
        self.Pile_Info = Pile_Info
        self.Layer_Properties = Layer_Properties
        self.Pile_Axial_Info = Pile_Axial_Info
    
    def Pile_Effective_Stress (self):
        self.top_pile = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Top Pile'])
        self.gs = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Ground Surface'])
        self.pile_length = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Pile Length'])
        self.bot_pile = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Top Pile']) - pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Pile Length'])
        
        self.pile_ruler = []
        for i in range(self.pile_length + 1):
            self.pile_ruler.append(self.top_pile - i)

        self.eff_stress = []
        for i in self.pile_ruler:
            if i > self.gs:
                self.eff_stress.append(0)
            
            # Boolean indexing to find the row where the variable fits between the upper and lower numbers
            matching_row = self.Layer_Properties[(self.Layer_Properties['mlv_top_elev'] <= i) & 
                                                (i < self.Layer_Properties['mlv_bottom_elev'])]


    def Nordlund_Method(self):

In [6]:
test_class = pile_design()

In [36]:
top_pile = pd.to_numeric(Pile_Axial_Info.iloc[0]['Top Pile'])
gs = pd.to_numeric(Pile_Axial_Info.iloc[0]['Ground Surface'])
pile_length = int(pd.to_numeric(Pile_Axial_Info.iloc[0]['Pile Length']))
bot_pile = pd.to_numeric(Pile_Axial_Info.iloc[0]['Top Pile']) - pd.to_numeric(Pile_Axial_Info.iloc[0]['Pile Length'])
        
pile_ruler = []

for i in range(pile_length + 1):
    pile_ruler.append(top_pile - i)

pile_ruler

[1,
 0,
 -1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 -8,
 -9,
 -10,
 -11,
 -12,
 -13,
 -14,
 -15,
 -16,
 -17,
 -18,
 -19,
 -20,
 -21,
 -22,
 -23,
 -24,
 -25,
 -26,
 -27,
 -28,
 -29,
 -30,
 -31,
 -32,
 -33,
 -34,
 -35,
 -36,
 -37,
 -38,
 -39,
 -40,
 -41,
 -42,
 -43,
 -44,
 -45,
 -46,
 -47,
 -48,
 -49,
 -50,
 -51,
 -52,
 -53,
 -54,
 -55,
 -56,
 -57,
 -58,
 -59,
 -60,
 -61,
 -62,
 -63,
 -64,
 -65,
 -66,
 -67,
 -68,
 -69,
 -70,
 -71,
 -72,
 -73,
 -74,
 -75,
 -76,
 -77,
 -78,
 -79,
 -80,
 -81,
 -82,
 -83,
 -84,
 -85,
 -86,
 -87,
 -88,
 -89,
 -90,
 -91,
 -92,
 -93,
 -94,
 -95,
 -96,
 -97,
 -98,
 -99]

In [51]:
Layer_Properties[['mlv_top_elev']] = Layer_Properties[['mlv_top_elev']].apply(pd.to_numeric)
Layer_Properties[['mlv_bottom_elev']] = Layer_Properties[['mlv_bottom_elev']].apply(pd.to_numeric)

i = int(-60)
# Boolean indexing to find the row where the variable fits between the upper and lower numbers
matching_row = Layer_Properties[(Layer_Properties['mlv_top_elev'] <= i) & 
                                                (i < Layer_Properties['mlv_bottom_elev'])]

matching_row

ValueError: Unable to parse string "nan" at position 0

In [49]:
Layer_Properties[['mlv_top_elev']] = Layer_Properties[['mlv_top_elev']].apply(pd.to_numeric)
Layer_Properties[['mlv_bottom_elev']] = Layer_Properties[['mlv_bottom_elev']].apply(pd.to_numeric)


In [50]:
print(Layer_Properties.dtypes)

layer                 object
non_soil              object
lv_top_elev          float64
mlv_top_elev         float64
hv_top_elev          float64
lv_bottom_elev       float64
mlv_bottom_elev       object
hv_bottom_elev       float64
lv_top_depth         float64
mlv_top_depth        float64
hv_top_depth         float64
lv_bottom_depth      float64
mlv_bottom_depth     float64
hv_bottom_depth      float64
lv_uw                float64
mlv_uw               float64
hv_uw                float64
lv_total_phi         float64
mlv_total_phi        float64
hv_total_phi         float64
lv_total_coh         float64
mlv_total_coh        float64
hv_total_coh         float64
lv_residual_phi      float64
mlv_residual_phi     float64
hv_residual_phi      float64
lv_residual_coh      float64
mlv_residual_coh     float64
hv_residual_coh      float64
lv_effective_phi     float64
mlv_effective_phi    float64
hv_effective_phi     float64
lv_effective_coh     float64
mlv_effective_coh    float64
hv_effective_c